In [54]:
from pyspark.sql import SparkSession

In [55]:
spark = SparkSession \
    .builder \
    .appName('Desafio Modulo 2 - ML') \
    .getOrCreate()

spark.version

'3.5.1'

In [56]:
clidata_df = spark.read.csv('/home/pericles/Spark-Projects/IGTI-DESM2/stroke_data.csv', header='True', inferSchema='True')

clidata_df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [63]:
mean_age = clidata_df.agg({'age': 'mean'}).collect()[0][0]
mean_avg_glucose_leve = clidata_df.agg({'avg_glucose_level': 'mean'}).collect()[0][0]
mean_bmi = clidata_df.agg({'bmi': 'mean'}).collect()[0][0]

In [65]:
clidata_df = clidata_df.fillna(mean_age, subset=['age'])
clidata_df = clidata_df.fillna(mean_avg_glucose_leve, subset=['avg_glucose_level'])
clidata_df = clidata_df.fillna(mean_bmi, subset=['bmi'])

In [66]:
clidata_df.describe().show()

+-------+------------------+------+------------------+-------------------+-------------------+------------+---------+--------------+------------------+------------------+---------------+-------------------+
|summary|                 0|gender|               age|       hypertension|      heart_disease|ever_married|work_type|Residence_type| avg_glucose_level|               bmi| smoking_status|             stroke|
+-------+------------------+------+------------------+-------------------+-------------------+------------+---------+--------------+------------------+------------------+---------------+-------------------+
|  count|             67135| 67135|             67135|              67135|              67135|       67135|    67135|         67135|             67135|             67135|          67135|              67135|
|   mean|           33568.0|  NULL| 51.95950845311693|0.16410218217025396|0.10142250688910405|        NULL|     NULL|          NULL|113.41439606762462| 29.16154047813857|  

In [58]:
clidata_df.select('gender', 'hypertension', 'heart_disease', 'ever_married').distinct().show()

+------+------------+-------------+------------+
|gender|hypertension|heart_disease|ever_married|
+------+------------+-------------+------------+
|  Male|           0|            0|          No|
|  Male|           0|            1|         Yes|
| Other|           0|            0|         Yes|
|  Male|           1|            0|         Yes|
|Female|           0|            1|         Yes|
|  Male|           1|            1|          No|
|Female|           0|            0|         Yes|
|Female|           0|            1|          No|
|  Male|           1|            1|         Yes|
|Female|           0|            0|          No|
|Female|           1|            1|         Yes|
|  Male|           0|            0|         Yes|
|Female|           1|            0|         Yes|
|  Male|           0|            1|          No|
|Female|           1|            0|          No|
|Female|           1|            1|          No|
|  Male|           1|            0|          No|
| Other|           0

In [59]:
clidata_df.select('work_type', 'Residence_type').distinct().show()

+-------------+--------------+
|    work_type|Residence_type|
+-------------+--------------+
|      Private|         Urban|
|     children|         Urban|
|     Govt_job|         Rural|
|Self-employed|         Rural|
|     Govt_job|         Urban|
|      Private|         Rural|
| Never_worked|         Rural|
|     children|         Rural|
| Never_worked|         Urban|
|Self-employed|         Urban|
+-------------+--------------+



In [60]:
clidata_df.select('smoking_status', 'stroke').distinct().show()

+---------------+------+
| smoking_status|stroke|
+---------------+------+
|   never_smoked|     0|
|formerly smoked|     1|
|         smokes|     1|
|formerly smoked|     0|
|   never_smoked|     1|
|         smokes|     0|
+---------------+------+



In [4]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder